# Setting up Relevant Dependencies
Load needed libraries.

In [178]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

# Data

*Flavor Text.* Peter Firsov works in the car loan division of a major commercial bank. He wants to use a data-driven strategy to identify borrowers who are likely to default on the automobile loan. 

The data includes the historical data for 400 customers containing information on 
- whether the customer defaulted ("Default"),
- corresponding loan-to-value ratio ("LTV", in %),
- FICO credit score ("FICO"), 
- and age ("Age").

Let's have a quick look at the data.

In [179]:
df = pd.read_csv("Default.csv")
df.head()

,Default,LTV,FICO,Age
0,0,70.50,705,61
1,0,73.91,586,20
2,0,80.33,779,67
3,0,91.06,764,55
4,0,68.45,776,32


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Default  400 non-null    int64  
 1   LTV      400 non-null    float64
 2   FICO     400 non-null    int64  
 3   Age      400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB


# Linear Probability Model
Let $y$ to be the value of `Default` in `df`.
Let $x_1,x_2,x_3$ be `LTV`, `FICO`, and `Age` in `df`, respectively.

We now wish to estimate $\mathbb{P}(y|(x_1,x_2,x_3))$ by the model
$$\mathbb{P}(y|(x_1,x_2,x_3)) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3,$$
which can be done using linear regression.

In [181]:
X = df[['LTV', 'FICO', 'Age']] # Define the predictors as the vector X
y = df['Default'] # The dependent variable

regr = linear_model.LinearRegression() # Set up regression
regr.fit(X, y) # Estimate probability of Y being 1 given X-values P(Y | LTV, FICO, Age)

print(regr.intercept_,regr.coef_[0],regr.coef_[1],regr.coef_[2])
print(r2_score(y,regr.predict(X)))

0.15171288714647319 0.009569102161206686 -0.0013079276093557672 0.001797406533930259
0.1947650829541656


We see that, approximating up to four decimal digits,
$$\beta_0 \approx 0.1517,$$
$$\beta_1 \approx 0.0096,$$
$$\beta_2 \approx -0.0013;$$
$$\beta_3 \approx 0.0018.$$
and so we have, according to our model,
$$\mathbb{P}(y|(x_1,x_2,x_3)) = 0.1517+0.0096 x_1 -0.0013x_2 + 0.0018x_3.$$

Thus, 
- If all the variables are zero, the probability of defaulting is $0.1517$.
- For every unit increase in `LTV`, the probability increases by $0.0096$.
- For every unit increase in `FICO`, the probability decreases by $0.0013$.
- For every unit increase in `Age`, the probability increases by $0.0018$.

Then, the value of $R^2$ is also 0.1948 (approximated to four decimal digits), which is surely not good.

For example, if $x_1=70$, $x_2=600$, $x_3=50$, we have that
$$\mathbb{P}(y|(x_1,x_2,x_3)=(70,600,50))\approx 0.12666.$$

In [182]:
print(regr.predict([[70, 600, 50]]))

[0.1266638]


C:\Users\remus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Now, let's try to see the probability of $y=1$ according to the model given that $x_1,x_2,x_3$ are assigned to their respective arithmetic means in the dataset.

In [183]:
m1 = np.mean(df['LTV'])
m2 = np.mean(df['FICO'])
m3 = np.mean(df['Age'])

print(regr.predict([[m1, m2, m3]]))

[0.0975]


C:\Users\remus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


# Logistic Regression
Let us make use of a logistic regression now.
For this model, we are going to also make use of the `AgeSquared` as another predictor, which is just the square of the `Age`.

Young working-class people have more reliable income because of their more stable employment and support from family, so they are typically less likely to default on a loan. However, as they get older, an increase in financial responsibilities (e.g., taxes, providing for family, health concerns, etc.) begin to impact their lives more significantly as they become less reliant on their parents, guardians, or other relatives. But once, people reach a certain age, the risk of defaulting decreases due to reaching the pinnacle of their career, attaining retirement benefits as they get much older, sometimes receiving monetary assistance from relatives, or even a decrease in overall financial burdens (though healthcare becomes a more significant concern as their bodies become more vulnerable to disease due to age).

So, by adding the additional quadratic term, we can hopefully account for the nonlinear effect of age.

In [184]:
df['AgeSquared'] = df['Age']**2

X2 = df[['LTV', 'FICO', 'Age', 'AgeSquared']]

X2.head()

,LTV,FICO,Age,AgeSquared
0,70.50,705,61,3721
1,73.91,586,20,400
2,80.33,779,67,4489
3,91.06,764,55,3025
4,68.45,776,32,1024


## Training the Model
Prepare for the logistic regression by training and testing the system defined by the data. This is to prevent overfitting. Once done, we can finally begin the fitting.

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=50) 
# we are only going to test 20% of the data points (i.e., 80 data points).
# random_state is set to 50 for the reproducibility of the results below.

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

## Relevant Information
Let's check some relevant information about the logistic regression model we had just made. We see that the accuracy is around 92.5%, which I'd say is not that bad. Let us also check the confusion matrix and the classification report.

In [186]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.925
Confusion Matrix:
 [[69  3]
 [ 3  5]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96        72
           1       0.62      0.62      0.62         8

    accuracy                           0.93        80
   macro avg       0.79      0.79      0.79        80
weighted avg       0.93      0.93      0.93        80



Based on the confusion matrix, we see that we have 3 false positives and 3 false negatives out of tested 80 data points.

Then, the classification report gives a more detailed look at the accuracy of our model.
- The model was correct 96% of the time when predicting `0` for the `Default` value (did not default the loan).
- However, the model was only correct 62% of the time when predicting `1` for the `Default` value (did default the loan).

However, there were 72 instances of `0` and only 8 instances of `1` in the test values. Thus, the poor performance in predicting `1`'s should be somewhat expected given the stark imbalance in the data.

In fact, notice the following

In [187]:
df['Default'].value_counts()

Default
0    361
1     39
Name: count, dtype: int64

So, given how values of `0` vastly dominate the dataset, the logit model would definitely have a hard time in training to correctly predict `1`'s.

## Prediction
Let us now observe how our model makes predictions. To begin, let us take a look if the following particular values of `LTV`, `FICO`, and `Age` would be indicative of defaulting on a loan.

In [188]:
# LTV=70, FICO=600, Age=50
print(model.predict([[70, 600, 50, 50*50]]))

# LTV=80, FICO=650, Age=40
print(model.predict([[80, 650, 40, 40*40]]))

[0]
[0]


C:\Users\remus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\remus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


To see why `0` was returned for both values, let us get the coefficients and use them to calculate the log odds and probability of `Default` being "1" given values for $x_1,x_2,x_3$ (`LTV`, `FICO`, and `Age` respectively).

In [189]:
print(model.intercept_,model.coef_)

model_coefficients = model.coef_[0]
model_intercept = model.intercept_[0]

def logodds(x1,x2,x3,x32):
    input_values = np.array([x1, x2, x3, x32])
    log_odds = model_intercept + np.dot(model_coefficients, input_values)
    return log_odds

def probability(x1,x2,x3,x32):
    probability = np.exp(logodds(x1,x2,x3,x32)) / (1 + np.exp(logodds(x1,x2,x3,x32)))
    return probability

print("Odds: " + str(np.exp(logodds(70,600,50,50*50))), "Probability: " + str(probability(70,600,50,50*50)))
print("Odds: " + str(np.exp(logodds(70,700,50,50*50))), "Probability: " + str(probability(70,700,50,50*50)))

[-29.32076717] [[ 0.23822489 -0.02928035  1.1828937  -0.01258808]]
Odds: 0.07896624835082355 Probability: 0.07318694951906211
Odds: 0.00422485952645978 Probability: 0.004207085182547666


Thus, with these coefficients, our logit model is given by the following:
$$\ln\left(\frac{\mathbb{P}(\mathrm{Default}=1) }{\mathbb{P}(\mathrm{Default}=0) }\right) = -29.32076717 + 0.23822489\cdot\mathrm{LTV} -0.02928035\cdot\mathrm{FICO} + 1.1828937\cdot\mathrm{Age} -0.01258808\cdot\mathrm{Age}^2  $$

We can thus interpret the model as follows:
- For every unit increase in `LTV` (keeping all else constant), log-odds increases by $0.2382$, i.e., the odds gets multiplied by $e^{0.2382}\approx 1.2690$ (around 26% increase in odds).
- For every unit increase in `FICO`, the log-odds decreases by $0.0293$, i.e., the odds gets multiplied by $e^{-0.0293}\approx 0.9711$ (around 3% decrease in odds).
- Now, letting $A$ denote `Age`, notice that $\frac{d}{dA}(\textrm{log-odds}) = 1.1828 - 2(0.0126 A)$. Thus, the log-odds of defaulting peak at an age of $\frac{1.1828}{2(0.0126)}\approx 46.9365$, i.e., at around 47 years old. So, risk of defaulting increases as a customer gets older, but it tends to decrease after 47 years old.

## Statistical Significance
Unfortunately, the `sklearn` module does not provide much functionality for that, so let us make use of the `statsmodels` module.

In [190]:
import statsmodels.api as sm

model2 = sm.Logit(y, sm.add_constant(X2))
result = model2.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.152182
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                Default   No. Observations:                  400
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            4
Date:                Sun, 11 Jan 2026   Pseudo R-squ.:                  0.5238
Time:                        19:33:41   Log-Likelihood:                -60.873
converged:                       True   LL-Null:                       -127.82
Covariance Type:            nonrobust   LLR p-value:                 5.708e-28
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -28.5195      7.238     -3.940      0.000     -42.706     -14.332
LTV            0.2278      0

First, we note that the pseudo R-squared value is 0.52, which somewhat good as it means that our predictors account for the majority (more than 50%) of the variation in the log-odds.
More importantly, notice how the $p$-values (under `P>|z|`) are all less than, for example, $0.001$, since they all display as `0.000`. This shows that each of these predictors are significant. Though the calculated coefficients differ, the $p$-values of our original model should not be too far off from what was calculated above.

In particular, for the `Age` variable, both the `Age` and `AgeSquared` have tiny associated $p$-values. This hints at statstically significant evidence of a non-linear relationship between `Age` and the log-odds of defaulting on a loan.